## **Распределительная задача**

Завод может производить Y = 25 видов продукции. На продукцию есть покупатель, которому требуется определенное количество каждого вида:

Таблица M (кг): M = [52 37 65 64 66 66 62 49 33 54 49 67 60 68 69 35 45 53 62 40 32 54 64 60 47]

Для каждого вида продукции он либо покупает требуемое количество, либо не покупает совсем, если мы произвели недостаточно. При этом известна цена контракта на каждый вид продукции:

Таблица C (рубли): C = [656 672 675 665 512 620 451 538 523 670 591 421 574 531 608 454 471 414 605 543 612 514 638 444 437]

Завод одновременно может выпускать только один вид продукции со скоростью:

Таблица P (кг в сутки): P = [19 15 27 19 29 16 22 18 16 27 21 21 24 25 28 21 24 21 15 15 17 29 27 24 21]

Требуется написать математическую модель и найти оптимальный план производства на S = 20 суток с помощью Gurobi, если известно, что переход от одной продукции к другой происходит мгновенно, а его стоимость R = 49 рублей.


#### Запишем математическую модель:

Введём переменные:

$$x_{i} = \begin{cases}
   1,&\text{если заключаем контракт с } i &\text{заводом,}\\
   0,&\text{в противном случае.}
\end{cases}$$

Целевая функция:

$$\sum\limits_{i = 0}^{Y} (c_i - R)x_i + R\to \max_{x}$$

Ограничения:

$$x_i \in \mathbb B, \forall i \in Y $$

$$\sum\limits_{i = 0}^{Y} x_i \lceil m_i/p_i \rceil \leqslant S $$

In [13]:
import pulp
import numpy as np

In [14]:
Y = 25
S = 20
R = 49

M = [52, 37, 65, 64, 66, 66, 62, 49, 33, 54, 49, 67, 60, 68, 69, 35, 45, 53, 62, 40, 32, 54, 64, 60, 47]
C = [656, 672, 675, 665, 512, 620, 451, 538, 523, 670, 591, 421, 574, 531, 608, 454, 471, 414, 605, 543, 612, 514, 638, 444, 437]
P = [19, 15, 27, 19, 29, 16, 22, 18, 16, 27, 21, 21, 24, 25, 28, 21, 24, 21, 15, 15, 17, 29, 27, 24, 21]

In [15]:
production_types = range(0, Y)

In [16]:
problem = pulp.LpProblem('Distribution_problem', pulp.LpMaximize)

factory_level_vars = pulp.LpVariable.dicts('factories_included', production_types, cat='Binary')
problem += pulp.lpSum([(C[i]- R) * factory_level_vars[i] for i in range(Y)]) + R, 'Total amount of money'

for i in production_types:
    problem += pulp.lpSum([factory_level_vars[i] * np.ceil(M[i] / P[i])]) <= S, 'Maximum amount of days ' + str(i)

In [20]:
status = problem.solve()
print(pulp.LpStatus[status])

for v in problem.variables():
    print(f'{v.name} = {v.varValue}')

print(f'Общая стоимость = {pulp.value(problem.objective)}')

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/qx/sdp1h2j15vzbw5kkm3jc_1_m0000gq/T/db738556e9c54eb3ae744eadc07aabfa-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/qx/sdp1h2j15vzbw5kkm3jc_1_m0000gq/T/db738556e9c54eb3ae744eadc07aabfa-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 30 COLUMNS
At line 131 RHS
At line 157 BOUNDS
At line 183 ENDATA
Problem MODEL has 25 rows, 25 columns and 25 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 12614 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from -12614 to -1.79769e+308
Probing was tried 0 times and